In [1]:
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser

In [2]:
import src.utils as utils
from src.Netlist.AST import AST
top="fsm_0_obf"
pathin="input_files/ASSURE_LOCKED/design1/design1_netlist.v"

# top="fsm"
# pathin="input_files/ASSURE_LOCKED/design1/oracle1_netlist.v"

# utils.clean_dir("./tmp")

obj=AST(file_path=pathin,rw="w",flag="v",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design



Verifiying Input Verilog File
Input Verilog File Verified Without Issue
extract_modules_def took 0.0015184879302978516 seconds to complete its execution.
Generating Logic Locking AST File
gates_module_extraction took 0.017525911331176758 seconds to complete its execution.
Done Generating Logic Locking AST File
Writing LL file
	 Netlist Not Locked, No Update to Peform
Done Writing LL file


In [2]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


extract_modules_def took 0.012199878692626953 seconds to complete its execution.
tmp2 took 24.10127568244934 seconds to complete its execution.
HERE


In [3]:
gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)

gates_module_extraction took 12.613260507583618 seconds to complete its execution.


In [4]:
# FF_tech

In [5]:
txt=utils.FF_to_txt(FF_tech,FF)

In [ ]:
# txt=gates_to_txt(gate_tech,gates)
with open("./tmp.v","w") as f:
  f.write(txt)